## Importing the librarys

In [ ]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten,Dropout

## Collecting Data set

- for training we have used 10k real images from imagenet dataset https://www.kaggle.com/datasets/lijiyu/imagenet
- and for AI generated Images we have used 10k images of stable-diffusion dataset https://www.kaggle.com/datasets/sahalmulki/stable-diffusion-generated-images

In [13]:
os.mkdir("./data")

In [14]:
os.mkdir("./data/fake")
os.mkdir("./data/real")

In [4]:
folder_path="/kaggle/input/stable-diffusion-generated-images/StableDiff/StableDiff/StableDiff"

In [5]:
file_names=os.listdir(folder_path)

In [6]:
fake_path="./data/fake"

In [7]:
for file in file_names:
    img=Image.open(folder_path+"/"+file)
    img.save(fake_path+"/"+file+".JPEG")

In [8]:
len(os.listdir("./data/fake"))

4952

In [9]:
folder_path="/kaggle/input/stable-diffusion-generated-images/laion400m-laion4.75/laion400m-laion4.75/laion400m-laion4.75+/laion400m-laion4.75+"

In [10]:
file_names=os.listdir(folder_path)

In [14]:
for file in file_names:
    try:
        img=Image.open(folder_path+"/"+file)
        if img.format == 'PNG':
            if img.mode != 'RGBA':
                img=img.convert("RGBA")
        img=img.convert("RGB")
        img.save(fake_path+"/"+file+".JPEG")
    except:
        pass

In [15]:
len(os.listdir(fake_path))

9997

In [16]:
folder_path="/kaggle/input/imagenet/imagenet/train"

In [17]:
file_names=os.listdir(folder_path)
fake_path="./data/real"

In [ ]:
i=0
for file in file_names:
    img=Image.open(folder_path+"/"+file)
    img.save(fake_path+"/"+file+".JPEG")
    i=i+1
    if i==9997:
        break

In [ ]:
len(os.listdir("./data/real"))

## Building Model
- we have used VGG16 as base model 


## Model Architecture
1. Resclaing layer
2. VGG16 base model
3. Flatten layer
4. Dropout 0.5
5. Dense layer with 512 nodes and activation ReLU
6. Dropout 0.2
7. Dense layer with 256 nodes and activation ReLU
8. Dropout 0.1
9. A Dense output layer with 2 nodes and without activation

In [ ]:
vgg_base=VGG16(weights='imagenet',include_top=False,input_shape=(256,256,3))

In [ ]:
model=Sequential(
    [
        layers.Rescaling(1/255.0),
        vgg_base,
        Flatten(),
        Dropout(0.5),
        layers.Dense(512,activation='relu'),
        Dropout(0.2),
        layers.Dense(256,activation='relu'),
        Dropout(0.10),
        layers.Dense(2),
    ]
)

In [ ]:
vgg_base.trainable=False

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [25]:
img_height,img_width=256,256
batch_size=256

In [ ]:
train_ds=keras.utils.image_dataset_from_directory("./data",image_size=(img_height,img_width),batch_size=batch_size)

## Training The Model

In [ ]:
model.fit(train_ds,batch_size=batch_size,epochs=50,verbose=2)

## Save the Model

In [ ]:
model.save("./save")

In [9]:
download_model()

## Load Model

In [11]:
model=keras.models.load_model('model.h5')

## Loading the testing data

- for testing we have used 670 real images from imagenet 
- and 670 images from midjourney art week of april 9-2023  https://www.kaggle.com/datasets/nikbearbrown/midjourney-art-week-of-april-9-2023

In [15]:
folder_path="/kaggle/input/midjourney-art-week-of-april-9-2023/Kaggle_Midjourney_Art_April_9_2023"

In [16]:
file_names=os.listdir(folder_path)

In [17]:
fake_path="./data/fake"

In [18]:
for file in file_names:
    try:
        img=Image.open(folder_path+"/"+file)
        if img.format == 'PNG':
            if img.mode != 'RGBA':
                img=img.convert("RGBA")
        img=img.convert("RGB")
        img.save(fake_path+"/"+file+".JPEG")
    except:
        pass

In [19]:
len(file_names)

673

In [22]:
folder_path="/kaggle/input/imagenet/imagenet/val"

In [23]:
file_names=os.listdir(folder_path)
fake_path="./data/real"

In [24]:
i=0
for file in file_names:
    img=Image.open(folder_path+"/"+file)
    img.save(fake_path+"/"+file+".JPEG")
    i=i+1
    if i==673:
        break

In [26]:
test_ds=keras.utils.image_dataset_from_directory("./data",image_size=(img_height,img_width),batch_size=batch_size)

Found 1346 files belonging to 2 classes.


In [27]:
loss,acc=model.evaluate(test_ds)

6/6 [==============================] - 32s 2s/step - loss: 0.5989 - accuracy: 0.8395


- almost 97% accuracy on training dataset 
- and about 84% accuracy on test data set
- we have deployed the project on streamlit https://rahim-khan-iitg.streamlit.app/
- you can find project files at https://github.com/rahim-khan-iitg/streamlit.git
as well as in the google drive
- kaggle link for this notebook https://www.kaggle.com/code/rahimkhan76/real-fake-vgg2/edit/run/134200299
- you can download trained model (h5 file) using the link https://rahimcdn.blob.core.windows.net/mycdn/model.h5